## Tensorflow for regression

In [1]:
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')
%run Functions.py

/home/jiashen/env3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train = pd.read_csv('data/train_c.csv')
test = pd.read_csv('data/test_c.csv')

In [69]:
train,test = get_additional_features(train,test)

## Use xgb to select several features

In [5]:
y_mean = np.mean(train.y)
y_train = train.y
xgb_params = {
        'n_trees': 520, 
        'eta': 0.0045,
        'max_depth': 4,
        'subsample': 0.93,
        'objective': 'reg:linear',
        'eval_metric': 'rmse',
        'base_score': y_mean, # base prediction = mean(target)
        'silent': True,
        'seed': 42
    }
    # NOTE: Make sure that the class is labeled 'class' in the data file
    
dtrain = xgb.DMatrix(train.drop('y', axis=1), y_train)
dtest = xgb.DMatrix(test)
model = xgb.train(xgb_params,dtrain,num_boost_round=1250)

In [6]:
Feature_Importance = get_feature_importance(model)

In [35]:
FI = Feature_Importance[Feature_Importance['Score']>100]

In [101]:
FI.shape

(43, 2)

In [102]:
Features = list(FI['Feature'])

In [103]:
features = ['X0',
                'X5',
                'X118',
                'X127',
                'X47',
                'X315',
                'X311',
                'X179',
                'X314',
                'X232',
                'X29',
                'X263',
                'X261']
FE_append = [each for each in features if each not in Features]
Features = Features+FE_append

In [39]:
len(Features)

50

In [106]:
Features.append('magic')

In [105]:
train,test = get_additional_features(train,test,magic=True)

## Build Neural Network

In [3]:
y_train = train.y
train = train.as_matrix()
y_train = np.transpose([y_train.as_matrix()])

In [7]:
y_train

array([[ 130.81],
       [  88.53],
       [  76.26],
       ..., 
       [ 109.22],
       [  87.48],
       [ 110.85]])

In [107]:
test_id = test['ID']

In [109]:
y_train = train.y
train = train[Features]
test = test[Features]
train = train.as_matrix()
y_train = np.transpose([y_train.as_matrix()])
test = test.as_matrix()

In [110]:
STEPS = 500
LEARNING_RATE = 0.0001
BETA = 0.01
DROPOUT = 0.5
RANDOM_SEED = 42
MAX_Y = 250
RESTORE = True
START = 0

In [111]:
# Training variables
IN_DIM = len(Features)
# Network Parameters - Hidden layers
n_hidden_1 = 400
n_hidden_2 = 200

In [112]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.01)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.03, shape=shape)
    return tf.Variable(initial)

In [113]:
def deep_network(inputs, keep_prob):
    # Input -> Hidden Layer
    w1 = weight_variable([IN_DIM, n_hidden_1])
    b1 = bias_variable([n_hidden_1])
    # Hidden Layer -> Hidden Layer
    w2 = weight_variable([n_hidden_1, n_hidden_2])
    b2 = bias_variable([n_hidden_2])
    # Hidden Layer -> Output
    w3 = weight_variable([n_hidden_2, 1])
    b3 = bias_variable([1])

    # 1st Hidden layer with dropout
    h1 = tf.nn.relu(tf.matmul(inputs, w1) + b1)
    h1_dropout = tf.nn.dropout(h1, keep_prob)
    # 2nd Hidden layer with dropout
    h2 = tf.nn.relu(tf.matmul(h1_dropout, w2) + b2)
    h2_dropout = tf.nn.dropout(h2, keep_prob)

    # Run sigmoid on output to get 0 to 1
    out = tf.nn.sigmoid(tf.matmul(h1_dropout, w3) + b3)

    # Loss function with L2 Regularization
    regularizers = tf.nn.l2_loss(w1) + tf.nn.l2_loss(w2) + tf.nn.l2_loss(w3)

    scaled_out = tf.multiply(out, MAX_Y)  # Scale output
    return inputs, out, scaled_out, regularizers

In [114]:

    # Create the model
    x = tf.placeholder(tf.float32, [None, IN_DIM])

    # Define loss and optimizer
    y_ = tf.placeholder(tf.float32, [None, 1])

    # Dropout on hidden layers
    keep_prob = tf.placeholder("float")

    # Build the graph for the deep net
    inputs, out, scaled_out, regularizers = deep_network(x, keep_prob)

    # Normal loss function (RMSE)
    loss = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(y_, scaled_out))))

    # Loss function with L2 Regularization
    loss = tf.reduce_mean(loss + BETA * regularizers)

    # Optimizer
    train_step = tf.train.AdamOptimizer(LEARNING_RATE).minimize(loss)

    total_error = tf.reduce_sum(tf.square(tf.subtract(y_, tf.reduce_mean(y_))))
    unexplained_error = tf.reduce_sum(tf.square(tf.subtract(y_, scaled_out)))
    accuracy = tf.subtract(1.0, tf.divide(unexplained_error, total_error))

    # Save model
    saver = tf.train.Saver(max_to_keep=5)

    with tf.Session() as sess:
        #if RESTORE:
        #    print('Loading Model...')
        #    ckpt = tf.train.get_checkpoint_state('./models/neural/')
        #    saver.restore(sess, ckpt.model_checkpoint_path)
        #else:
        sess.run(tf.global_variables_initializer())

        train, y_train, test_submit, test_submit_id = get_data()

        # Train until maximum steps reached or interrupted
        for i in range(START, STEPS):
            k_fold = KFold(n_splits=10, shuffle=True)
            #if i % 100 == 0:
            #    saver.save(sess, './models/neural/step_' + str(i) + '.cptk')

            for k, (ktrain, ktest) in enumerate(k_fold.split(train, y_train)):
                train_step.run(feed_dict={x: train[ktrain], y_: y_train[ktrain], keep_prob: DROPOUT})
                # Show test score every 10 iterations
                if i % 10 == 0:
                    # Tensorflow R2
                    #train_accuracy = accuracy.eval(feed_dict={
                    #    x: train[ktest], y_: y_train[ktest]})
                    # SkLearn metrics R2
                    train_accuracy = r2_score(y_train[ktest],
                                              sess.run(scaled_out, feed_dict={x: train[ktest], keep_prob: 1.0}))
                    print('Step: %d, Fold: %d, R2 Score: %g' % (i, k, train_accuracy))

        ####################
        # CV (repeat 5 times)
        ####################
        CV = []
        for i in range(5):
            k_fold = KFold(n_splits=10, shuffle=True)
            for k, (ktrain, ktest) in enumerate(k_fold.split(train, y_train)):
                # Tensorflow R2
                #accuracy = accuracy.eval(feed_dict={
                #    x: train[ktest], y_: y_train[ktest]})
                # SkLearn metrics R2
                accuracy = r2_score(y_train[ktest],
                                          sess.run(scaled_out, feed_dict={x: train[ktest], keep_prob: 1.0}))
                print('Step: %d, Fold: %d, R2 Score: %g' % (i, k, accuracy))
                CV.append(accuracy)
        print('Mean R2: %g' % (np.mean(CV)))

    
    for i in range(5):
        if i==0:
            tf_train = pd.DataFrame(fold_train[i])
            label = [each[0] for each in fold_label[i].tolist()]
            pred = [each[0] for each in fold_pred[i].tolist()]
            tf_train.columns = Features
            tf_train['label'] = label
            tf_train['predicted'] = pred
        else:
            tmp = pd.DataFrame(fold_train[i])
            label = [each[0] for each in fold_label[i].tolist()]
            pred = [each[0] for each in fold_pred[i].tolist()]
            tmp.columns = Features
            tmp['label'] = label
            tmp['predicted'] = pred
            tf_train = pd.concat([tf_train,tmp])
    pred = [each[0] for each in preds.tolist()]
    sub = pd.DataFrame({'ID':test_id,'y':pred})

Step: 0, Fold: 0, R2 Score: -2.19524
Step: 0, Fold: 1, R2 Score: -0.801223
Step: 0, Fold: 2, R2 Score: -1.12158
Step: 0, Fold: 3, R2 Score: -2.10815
Step: 0, Fold: 4, R2 Score: -1.68488
Step: 10, Fold: 0, R2 Score: -1.57799
Step: 10, Fold: 1, R2 Score: -1.069
Step: 10, Fold: 2, R2 Score: -1.3013
Step: 10, Fold: 3, R2 Score: -1.18356
Step: 10, Fold: 4, R2 Score: -1.41077
Step: 20, Fold: 0, R2 Score: -0.347846
Step: 20, Fold: 1, R2 Score: -0.49994
Step: 20, Fold: 2, R2 Score: -0.514091
Step: 20, Fold: 3, R2 Score: -0.399731
Step: 20, Fold: 4, R2 Score: -0.444281
Step: 30, Fold: 0, R2 Score: -0.122087
Step: 30, Fold: 1, R2 Score: -0.112959
Step: 30, Fold: 2, R2 Score: -0.152087
Step: 30, Fold: 3, R2 Score: -0.23053
Step: 30, Fold: 4, R2 Score: -0.239689
Step: 40, Fold: 0, R2 Score: -0.0754703
Step: 40, Fold: 1, R2 Score: 0.0439002
Step: 40, Fold: 2, R2 Score: -0.0649561
Step: 40, Fold: 3, R2 Score: -0.0598602
Step: 40, Fold: 4, R2 Score: -0.0120865
Step: 50, Fold: 0, R2 Score: 0.0454552
S

Step: 420, Fold: 2, R2 Score: 0.684829
Step: 420, Fold: 3, R2 Score: 0.568157
Step: 420, Fold: 4, R2 Score: 0.639845
Step: 430, Fold: 0, R2 Score: 0.623229
Step: 430, Fold: 1, R2 Score: 0.629817
Step: 430, Fold: 2, R2 Score: 0.648616
Step: 430, Fold: 3, R2 Score: 0.505929
Step: 430, Fold: 4, R2 Score: 0.652055
Step: 440, Fold: 0, R2 Score: 0.556818
Step: 440, Fold: 1, R2 Score: 0.647439
Step: 440, Fold: 2, R2 Score: 0.592757
Step: 440, Fold: 3, R2 Score: 0.624266
Step: 440, Fold: 4, R2 Score: 0.646945
Step: 450, Fold: 0, R2 Score: 0.627196
Step: 450, Fold: 1, R2 Score: 0.622278
Step: 450, Fold: 2, R2 Score: 0.515056
Step: 450, Fold: 3, R2 Score: 0.64671
Step: 450, Fold: 4, R2 Score: 0.627717
Step: 460, Fold: 0, R2 Score: 0.548269
Step: 460, Fold: 1, R2 Score: 0.662863
Step: 460, Fold: 2, R2 Score: 0.415856
Step: 460, Fold: 3, R2 Score: 0.559166
Step: 460, Fold: 4, R2 Score: 0.530192
Step: 470, Fold: 0, R2 Score: 0.656247
Step: 470, Fold: 1, R2 Score: 0.63371
Step: 470, Fold: 2, R2 Scor

In [115]:
save_results(tf_train,sub,'Tensorflow_400_200_layers_selected_features_plus_magic.csv')

In [ ]:
400/200 0.598488
400/200_All 0.584157
600/200_All 0.567697
300/150_All 0.524824
800/400_All Interupted
400/200_All Dropout 0.4 0.572777
400/200_All Dropout 0.6 0.570832
400/200 Selected Dropout 0.5 0.621843

In [108]:
tf_train

,ID,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,ica_11,grp_11,srp_11,tsvd_12,pca_12,ica_12,grp_12,srp_12,label,predicted
0,13.0,24.0,23.0,38.0,5.0,3.0,14.0,3.0,13.0,0.0,...,-0.010211,-3.322663,-3.816067,1.096843,-1.617561,-0.039583,-22.018516,15.264267,78.02,80.488861
1,27.0,49.0,20.0,19.0,4.0,3.0,12.0,8.0,7.0,0.0,...,0.021121,-8.130342,-6.360111,0.221450,-0.174829,0.010208,-36.200175,10.176178,108.67,114.569809
2,30.0,36.0,3.0,17.0,2.0,3.0,12.0,0.0,4.0,0.0,...,-0.007257,-2.214733,-3.816067,1.307101,-1.186497,-0.013207,-31.061471,11.448200,126.99,120.916832
3,44.0,32.0,4.0,37.0,2.0,3.0,16.0,3.0,4.0,0.0,...,0.006793,6.656428,-5.088089,-0.209871,-0.720437,-0.032389,-39.270676,17.808312,89.81,96.591515
4,47.0,40.0,0.0,11.0,5.0,3.0,16.0,8.0,18.0,0.0,...,-0.008185,5.718078,-5.088089,-0.083302,-0.462061,-0.006637,-36.259417,16.536289,90.81,90.446739
5,49.0,46.0,1.0,19.0,3.0,3.0,16.0,9.0,18.0,0.0,...,0.014337,1.982322,-5.088089,0.192305,-0.230450,0.002992,-43.268949,19.080334,94.57,93.370422
6,60.0,41.0,20.0,37.0,2.0,3.0,16.0,8.0,21.0,0.0,...,0.012419,1.569708,-5.088089,-1.091529,0.261533,-0.019149,-55.891597,17.808312,93.59,95.121635
7,62.0,46.0,1.0,19.0,2.0,3.0,16.0,3.0,1.0,0.0,...,0.017541,2.569142,-5.088089,-1.183595,1.357544,0.017734,-46.910314,17.808312,89.37,94.586182
8,109.0,50.0,3.0,37.0,2.0,3.0,16.0,9.0,13.0,0.0,...,0.006790,12.069918,-2.544045,0.136198,-0.178138,-0.009909,-78.411757,20.352356,110.52,116.334869
9,119.0,18.0,11.0,9.0,1.0,3.0,16.0,3.0,8.0,0.0,...,-0.017051,8.387996,-3.816067,-0.461001,0.625188,0.007748,-56.518646,19.080334,108.00,113.260612
